In [0]:
#https://www.tensorflow.org/tutorials/keras/save_and_load

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
import numpy as np

TensorFlow 2.x selected.


In [0]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format
import os

In [4]:
#Get an example dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


11493376/11490434 [==============================] - 0s 0us/step


In [5]:
# Define a simple sequential model
#this is to be able to save its results later
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#Save checkpoints during training
You can use a trained model without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The tf.keras.callbacks.ModelCheckpoint callback allows to continually save the model both during and at the end of training.

In [7]:
#Checkpoint callback usage
#Create a tf.keras.callbacks.ModelCheckpoint callback that saves weights only during training:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.0327 - accuracy: 1.0000
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 235us/sample - loss: 0.0324 - accuracy: 0.9990 - val_loss: 0.4213 - val_accuracy: 0.8690
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.0280 - accuracy: 0.9987
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 237us/sample - loss: 0.0270 - accuracy: 0.9990 - val_loss: 0.4033 - val_accuracy: 0.8760
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.0231 - accuracy: 0.9987
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 235us/sample - loss: 0.0237 - accuracy: 0.9990 - val_loss: 0.4100 - val_accuracy: 0.8750
Epoch 4/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.0200 - accuracy: 1.0000
Epoch

In [23]:
!ls {checkpoint_dir} #Check whats in checkpoint dir
#NOTA: AL PARECER EL !ls es lo quehace que aparesca el folder en tus files



checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0026.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0026.ckpt.index
cp-0001.ckpt.data-00000-of-00001  cp-0027.ckpt.data-00000-of-00001
cp-0001.ckpt.index		  cp-0027.ckpt.index
cp-0002.ckpt.data-00000-of-00001  cp-0028.ckpt.data-00000-of-00001
cp-0002.ckpt.index		  cp-0028.ckpt.index
cp-0003.ckpt.data-00000-of-00001  cp-0029.ckpt.data-00000-of-00001
cp-0003.ckpt.index		  cp-0029.ckpt.index
cp-0004.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0004.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0031.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0031.ckpt.index
cp-0006.ckpt.data-00000-of-00001  cp-0032.ckpt.data-00000-of-00001
cp-0006.ckpt.index		  cp-0032.ckpt.index
cp-0007.ckpt.data-00000-of-00001  cp-0033.ckpt.data-00000-of-00001
cp-0007.ckpt.index		  cp-0033.ckpt.index
cp-0008.ckpt.data-00000-of-00001  cp-0034.ckpt.data-00000-of-00001
cp-0008.ckpt.index		  cp-0034.ckpt.

# Load the weights
Create a new, untrained model. When restoring a model from weights-only, you must have a model with the same architecture as the original model. Since it's the same model architecture, you can share weights despite that it's a different instance of the model.

In [10]:
#Now rebuild a fresh, untrained model, and evaluate it on the test set.
#An untrained model will perform at chance levels (~10% accuracy):
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))



1000/1000 - 0s - loss: 2.3160 - accuracy: 0.1170
Untrained model, accuracy: 11.70%


In [11]:
#Then load the weights from the checkpoint and re-evaluate:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 - 0s - loss: 0.4513 - accuracy: 0.8640
Restored model, accuracy: 86.40%


#Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [0]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5000) #period is save every 5 epochs. En warnings me dijo que es mejor usar save_freq.
    #Pero no estoy seguro cada cuanto representa. creo que escada 5 batchs??

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)


In [14]:
!ls {checkpoint_dir}


checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0026.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0026.ckpt.index
cp-0001.ckpt.data-00000-of-00001  cp-0027.ckpt.data-00000-of-00001
cp-0001.ckpt.index		  cp-0027.ckpt.index
cp-0002.ckpt.data-00000-of-00001  cp-0028.ckpt.data-00000-of-00001
cp-0002.ckpt.index		  cp-0028.ckpt.index
cp-0003.ckpt.data-00000-of-00001  cp-0029.ckpt.data-00000-of-00001
cp-0003.ckpt.index		  cp-0029.ckpt.index
cp-0004.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0004.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0031.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0031.ckpt.index
cp-0006.ckpt.data-00000-of-00001  cp-0032.ckpt.data-00000-of-00001
cp-0006.ckpt.index		  cp-0032.ckpt.index
cp-0007.ckpt.data-00000-of-00001  cp-0033.ckpt.data-00000-of-00001
cp-0007.ckpt.index		  cp-0033.ckpt.index
cp-0008.ckpt.data-00000-of-00001  cp-0034.ckpt.data-00000-of-00001
cp-0008.ckpt.index		  cp-0034.ckpt.

In [15]:
#Check latest training checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [16]:
#To test, reset the model and load the latest checkpoint:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 - 0s - loss: 0.4813 - accuracy: 0.8760
Restored model, accuracy: 87.60%


#Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the Model.save_weights method. By default, tf.keras—and save_weights in particular—uses the TensorFlow checkpoints format with a .ckpt extension (saving in HDF5 with a .h5 extension is covered in the Save and serialize models guide):

In [17]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 - 0s - loss: 0.4813 - accuracy: 0.8760
Restored model, accuracy: 87.60%


#Save the entire model

Call model.save to save the a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

In [18]:
#It is saved in the hd5 format
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model shuold be saved to HDF5.
model.save('my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 362us/sample - loss: 1.1443 - accuracy: 0.6680
Epoch 2/5
1000/1000 [==============================] - 0s 133us/sample - loss: 0.4208 - accuracy: 0.8760
Epoch 3/5
1000/1000 [==============================] - 0s 138us/sample - loss: 0.3006 - accuracy: 0.9210
Epoch 4/5
1000/1000 [==============================] - 0s 130us/sample - loss: 0.2042 - accuracy: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 129us/sample - loss: 0.1522 - accuracy: 0.9680


In [19]:
#Recrate model from that file
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
#Check loaded model accuracy
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))


1000/1000 - 0s - loss: 0.4094 - accuracy: 0.8640
Restored model, accuracy: 86.40%


In [21]:
"""
SavedModel format
The SavedModel format is another way to serialize models. 
Models saved in this format can be restored using tf.keras.models.load_model and
are compatible with TensorFlow Serving. 
The SavedModel guide goes into detail about how to serve/inspect the SavedModel. 
The section below illustrates the steps to saving and restoring the model.
"""
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 359us/sample - loss: 1.1613 - accuracy: 0.6710
Epoch 2/5
1000/1000 [==============================] - 0s 127us/sample - loss: 0.4237 - accuracy: 0.8810
Epoch 3/5
1000/1000 [==============================] - 0s 132us/sample - loss: 0.2977 - accuracy: 0.9300
Epoch 4/5
1000/1000 [==============================] - 0s 130us/sample - loss: 0.2161 - accuracy: 0.9390
Epoch 5/5
1000/1000 [==============================] - 0s 130us/sample - loss: 0.1599 - accuracy: 0.9640
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [22]:
#inspect saved model format
# my_model directory
!ls saved_model #NOTA: AL PARECER EL !ls es lo quehace que aparesca el folder en tus files

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [24]:
#Get a fresh keras model from the saved model
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)


1000/1000 - 0s - loss: 0.4139 - accuracy: 0.8640
Restored model, accuracy: 86.40%
(1000, 10)


NOTE: THE hd5 format is different from the saveModel and the following must be considered:
The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the orginal code.

To save custom objects to HDF5, you must do the following:

1. Define a get_config method in your object, and optionally a from_config classmethod:

  get_config(self) returns a JSON-serializable dictionary of parameters needed to recreate the object.

  from_config(cls, config) uses the returned config from get_config to create a new object. By default, this function will use the config as initialization kwargs (return cls(**config)).

2. Pass the object to the custom_objects argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})
